<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/notebooks/freeze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
workdir = '/content/workspace/MyDrive/tensorflow/power-port-targeting/'
modeldir = workdir + 'exported-models/trained_model/'
model = modeldir + 'saved_model'
from google.colab import drive
import os
import tensorflow as tf
!mkdir -p /content/workspace
drive.mount('/content/workspace')
print(tf.__version__)

Drive already mounted at /content/workspace; to attempt to forcibly remount, call drive.mount("/content/workspace", force_remount=True).
2.8.0


In [2]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.python.tools import freeze_graph

def freeze(model, dir, name):
    m = tf.saved_model.load(model)

    tfm = tf.function(lambda x: m(x))                                               
    tfm = tfm.get_concrete_function(
        tf.TensorSpec(
            m.signatures['serving_default'].inputs[0].shape.as_list(), 
            m.signatures['serving_default'].inputs[0].dtype.name
        )
    )   
    frozen_func = convert_variables_to_constants_v2(tfm)                                                                                                                              
    tf.io.write_graph(
        graph_or_graph_def=frozen_func.graph,
        logdir=dir, 
        name=name, 
        as_text=False
    )
    

In [3]:
freeze(model, modeldir, 'frozen_graph.pb')